In [1]:
import langid


import codecs
import pandas as pd
import re
#nltk.download('stopwords')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
#nltk.download('punkt')
import spacy


import json
import sys
import pathlib
from pathlib import Path
import os

In [2]:
def decode(path: str) -> list:
    f = codecs.open(path, errors = 'ignore', encoding = 'utf-8')
    data = f.read()
    data = data.encode('latin1').decode('utf8')
    data = data.splitlines()  
    return data

In [3]:
def change_letter(dataset: pd.DataFrame) -> list:
    new_dataset = [re.sub('ø', 'oe', text) for text in dataset]
    new_dataset = [re.sub('æ', 'ae', text) for text in new_dataset]
    new_dataset = [re.sub('å', 'aa', text) for text in new_dataset]
    new_dataset = [re.sub('Ø', 'oe', text) for text in new_dataset]
    new_dataset = [re.sub('Æ', 'ae', text) for text in new_dataset]
    new_dataset = [re.sub('Å', 'aa', text) for text in new_dataset]
    new_dataset = [re.sub('ü', 'ue', text) for text in new_dataset]
    new_dataset = [re.sub('Ü', 'ue', text) for text in new_dataset]
    new_dataset = [re.sub('ä', 'ae', text) for text in new_dataset]
    new_dataset = [re.sub('Ä', 'ae', text) for text in new_dataset]
    new_dataset = [re.sub('ö', 'oe', text) for text in new_dataset]
    new_dataset = [re.sub('Ö', 'oe', text) for text in new_dataset]
    
    return new_dataset

In [4]:
def get_clean_text(dataset: list, stops_da, stops_en) -> list:
    no_urls = [re.sub(r"http\S+", "", text) for text in dataset] 
    only_letters = [re.sub(r"(#[A-Za-z]+)|(@[A-Za-z]+)|([^A-Za-z \t])|(\w+:\/\/\S+)", ' ' , text) for text in no_urls] 
    only_letters = [text.replace('\n', ' ') for text in only_letters]
    lowercased_str = [text.lower().split() for text in only_letters] 
    no_stopwords = [[w for w in text if not w in stops_da] for text in lowercased_str] 
    no_stopwords = [[w for w in text if not w in stops_en] for text in no_stopwords] 
    cleaned_text = [" ".join(text) for text in no_stopwords] 
    return cleaned_text

In [5]:
def tokenize_text(data):
    tokens = [word_tokenize(text) for text in data] 
    return tokens

In [6]:
def lemmatize_text(sent):
    lemmas = [x.lemma_ for x in nlp(sent)]
    return lemmas

In [7]:
def lemmatize_posts(tokenized_data: list):
    lemmas = []
    for post in tokenized_data:
        lemma = [lemmatize_text(x) for x in post]
        lemmas.append([item for sublist in lemma for item in sublist])
    return lemmas

In [8]:
def remove_mentions(dataset: list):
    new_dataset = []
    for i in dataset:
        if i.startswith('@'):
          i = i.replace(i, "")
        new_dataset.append(i)
    return new_dataset

In [9]:
def only_da(dataset: list) -> list:

    langid.set_languages(['da', 'en'])  
    da = []

    for i in dataset:
        lang = langid.classify(i)
        if lang[0] != 'en':          
            da.append(i)


    return da

In [ ]:
file = open("../stop_words.txt","r+")
stop_words = file.read().split()

In [10]:
decoded_data = decode('prepared_txt/all_data.txt')
print(len(decoded_data))

19596


In [11]:
new_data = remove_mentions(decoded_data)

print(len(new_data))

19596


In [12]:
without_empty_strings = [string for string in new_data if string != ""]

print(len(without_empty_strings))

19390


In [13]:
fb_posts = change_letter(without_empty_strings)


[':proever at finde ud af at bruge facebook  - kraever lige lidt mere tid end jeg har nu.', 'sikken dejlig weekend - byggede hestehus og var ude at ride i det gode vejr loerdag. Soendag: ridetur og hestehusbyggeri i oesende regnvejr!!!']


In [71]:
no_urls = [re.sub(r"http\S+", "", text) for text in fb_posts] 
only_letters = [re.sub(r"(#[A-Za-z]+)|(@[A-Za-z]+)|([^A-Za-z \t])|(\w+:\/\/\S+)", ' ' , text) for text in no_urls]

In [72]:
splitted_text = [text.split() for text in only_letters] 
joined_text = [" ".join(text) for text in splitted_text] 

In [73]:
remove_empty_strings = [string for string in joined_text if string != ""] 
print(len(remove_empty_strings))

18627


In [74]:
da = only_da(remove_empty_strings)

print(len(da))

15145


In [20]:
#def translate_post(dataset: list) -> list:
#    translator = Translator()
#    langid.set_languages(['da', 'en'])  
#    count = 0
#    translated = []
#
#    for i in dataset:
#        lang = langid.classify(i)
#        if lang[0] == 'en':          
#            t = (translator.translate(i, dest = 'da'))
#            translated.append(t.text)
#            
#
#        else:
#            translated.append(i)
#         
#        
#    return translated


In [50]:
#translated_posts = translate_post(remove_empty_strings[:10000])

In [82]:
nlp = spacy.load("da_core_news_sm")
stopwords_da = stopwords.words("danish")  
stopwords_da.extend(stop_words)

In [26]:
nlp1 = spacy.load("en_core_web_sm")
stopwords_en = stopwords.words("english")

In [83]:
cleaned_posts = get_clean_text(da, stops_da = stopwords_da, stops_en = stopwords_en)

In [84]:
no_empty_strings = [string for string in cleaned_posts if string != ""]

In [ ]:
df1 = pd.DataFrame(no_empty_strings , columns=['cleaned_posts'])

In [85]:
tokenized_posts = tokenize_text(no_empty_strings)

In [88]:
lemmatized_posts = lemmatize_posts(tokenized_posts)

In [89]:
df1['tokenized_posts'] = tokenized_posts
df1['lemmatized_posts'] = lemmatized_posts


In [95]:
#df1.to_pickle('prepared_txt/all_posts_da_df.pkl')